In [36]:
import os, sys
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib
import matplotlib.pyplot as plt
import yaml

from getdist import loadMCSamples, plots, mcsamples, MCSamples
from tensiometer import utilities
from tensiometer import gaussian_tension

chain_dir = os.path.join(os.getcwd(), 'chains')

prior = loadMCSamples(file_root=chain_dir+'/lcdm_eftboss/2023-10-12_10000_cosmo_prior_', 
                          settings={'ignore_rows':0., 'smooth_scale_1D':-1, 'smooth_scale_2D':-1})
prior.name_tag = 'BOSS Prior'
# Check completeness
print(f'List of \"cosmological\" parameters: {prior.getParamNames().getRunningNames()}')
print(f'     List of \"derived\" parameters: {prior.getParamNames().getDerivedNames()}')

List of "cosmological" parameters: ['omega_b', 'omega_cdm', 'h', 'ln10^{10}A_s', 'n_s']
     List of "derived" parameters: ['Omega_m', 'A_s', 'sigma8']


In [37]:
# Repeat the process for the posterior
posterior = loadMCSamples(file_root=chain_dir+'/lcdm_eftboss/2023-10-12_10000_',
                          settings={'ignore_rows':0.3, 'smooth_scale_1D':-1, 'smooth_scale_2D':-1})
posterior.name_tag = 'BOSS'

In [38]:
from tensiometer import utilities
from tensiometer import gaussian_tension

# Re-definition of cosmo_params to match the Tensiometer example
params = ['h', 'omega_cdm', 'n_s', 'ln10^{10}A_s', 'Omega_m', 'sigma8']
labels = ['h', '\omega_{cdm}', 'n_s', '\ln{10^{10}A_s}', '\Omega_m', '\sigma_8']

print('N_eff = ', gaussian_tension.get_Neff(posterior, prior_chain=prior, param_names=params))

N_eff =  4.37759806706756


In [39]:
# Checkpoint: run this cell to refresh the chains
prior_chain = prior.copy()
posterior_chain = posterior.copy()

# Take the logarithm of the parameters: KL decomposition -> power-law decomposition
log_params = []
log_labels = []
for name, label in zip(params, labels):
    for chain in [prior_chain, posterior_chain]:
        p = chain.getParams()
        method = getattr(p, name)
        chain.addDerived(np.log(method), name='log_'+name, label='\\log '+label)
    log_params += ['log_'+name]
    log_labels += ['\\log '+label]

print('N_eff = ', gaussian_tension.get_Neff(posterior_chain, prior_chain=prior_chain, param_names=log_params))

N_eff =  4.0021382901837805


In [40]:
# Compute the KL modes
KL_param_names = log_params
KL_eig, KL_eigv, KL_param_names = gaussian_tension.Q_UDM_KL_components(prior_chain, posterior_chain, 
                                                                    param_names=KL_param_names)

with np.printoptions(precision=2, suppress=True):
    if any(eig < 1. for eig in KL_eig):
        print('Improvement factor over the prior:', KL_eig)
        print('Discarding error units due to negative values.')
    else:
        print('Improvement factor over the prior in E.U.:', np.sqrt(KL_eig-1))

Improvement factor over the prior: [53.47 15.51  9.25  5.19  1.63  0.81]
Discarding error units due to negative values.


In [53]:
# Compute the fractional Fisher Matrix
KL_param_names, KL_eig, fractional_fisher, _ = gaussian_tension.Q_UDM_fisher_components(prior_chain, posterior_chain, 
                                                                                        params, 
                                                                                        which='12')
# Columns do not sum up to 1
temp_row = 0
temp_col = 0
for i in range(len(fractional_fisher)):
    print(f'Row {i}: {sum(fractional_fisher[i,:]):.2}', end='\t')
    temp_row += sum(fractional_fisher[i,:])
    print(f'Col {i}: {sum(fractional_fisher[:,i]):.2}')
    temp_col += sum(fractional_fisher[:,i])
print(f'Sum = {temp_row:.2}\tSum = {temp_col:.2}')

Row 0: 1.0	Col 0: 0.021
Row 1: 1.0	Col 1: 1.3
Row 2: 1.0	Col 2: 0.068
Row 3: 1.0	Col 3: 0.63
Row 4: 1.0	Col 4: 0.19
Row 5: 1.0	Col 5: 0.019
Sum = 6.0	Sum = 6.0
